In [28]:
# combine sentiment by stanford
import pandas as pd
import numpy as np
df = pd.read_csv('sentiment_tweet.csv', quotechar='"')
del df['text']
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['user']

import ast

def getSent(row):
    sentiment = ast.literal_eval(row['sentiment'])['sentiment']
    stanford = row['stanford']
    if stanford == 1:
        stanford = 1
    if stanford == -1:
        stanford = -1
    if stanford == 2:
        stanford = 1
    if stanford == -2:
        stanford = -1
    if sentiment == None:
        return stanford
    else:
        if sentiment['basic'] == 'Bullish':
            return 2
        if sentiment['basic'] == 'Bearish':
            return -2
        return stanford
    
df['sentiment'] = df.apply(getSent, axis=1)
del df['stanford']
df.head()
df['sentiment'].value_counts()

-1    81288
 2    63777
 0    41977
-2    25351
 1    10583
Name: sentiment, dtype: int64

In [30]:
# aggregate by date
def getIndex(arr):
    bull = 0
    bear = 0
    for i in arr:
        if i > 0:
            bull += i
        if i < 0:
            bear += -i
#     return bull-bear
    if bull == bear:
        return 0
    elif bull > bear:
        return ((2*bull)/(bull+bear) - 1) / len(arr)
    else:
        return (1 - (2*bear)/(bull+bear)) / len(arr)

data = df.groupby('date').agg({'sentiment': getIndex})['sentiment']
data.to_csv('aggregate_tweet.csv')
data.head()

date
2018.03.16   -0.005629
2018.03.17   -0.000658
2018.03.18   -0.000340
2018.03.19   -0.000106
2018.03.20   -0.000543
Name: sentiment, dtype: float64

In [48]:
#add time series to sentiment
import datetime as dt
df = pd.read_csv("./AAPL_data.csv", usecols=[0,1,2,3,4,5])
df['date'] = df['date'].map(lambda x: dt.datetime.strptime(x,"%Y-%m-%d").strftime('%Y.%m.%d'))

data = pd.read_csv('./aggregate_tweet.csv', header=None, names = ['date', 'sentiment'])
data = data.set_index('date')
df = df.set_index('date')
result = pd.concat([data, df], axis=1, join='inner')
result['change'] = df['close'] - df['open']
result.to_csv('./merged_result.csv')
result = pd.read_csv('./merged_result.csv', usecols=[1,5,6,7])
result.head()

result['output'] = result.close.shift(-1)

result.dropna(inplace=True)
result.reset_index(drop=True,inplace=True)
try:
    result = result.drop(columns=['date'])
except:
    pass

In [50]:
#preprocess dataframe
up = 1
down = -1
def setLabel(row):
    yesterday = row['close']
    today = row['output']
    
    #change = today - yesterday
#     return today
    return today - yesterday
    if yesterday < today:
        return up
    if today < yesterday:
        return down
    return 0

result['output'] = result.apply(setLabel, axis=1) #soal

def norm(a):
    b = a - a.min()
    return b / b.max()

result['sentiment'] = norm(result['sentiment'])
result['output'] = norm(result['output'])
result['change'] = norm(result['change'])
result['close'] /= result['close'].max()
result['volume'] /= result['volume'].max()
result.to_csv('./ML_cols.csv', encoding='utf-8')
result.head()

,sentiment,close,volume,change,output
0,0.000000,0.761479,0.409413,0.529666,0.579368
1,0.847330,0.749844,0.347511,0.443428,0.678926
2,0.780347,0.749587,0.204156,0.568751,0.547805
3,0.852223,0.732606,0.384999,0.334851,0.614765
4,0.864434,0.722254,0.431087,0.497403,0.573786


In [54]:
#fit model

import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU
from keras.layers import LSTM, Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_dataset1(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = []
        for j in range(look_back):
            a += dataset[i+j, :-1].tolist()
        dataX.append(a)
        dataY.append(dataset[i + look_back, -1])
    return numpy.array(dataX), numpy.array(dataY)

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(look_back-1, len(dataset)):
        a = []
        for j in range(look_back):
            a += dataset[i-j, :-1].tolist()
        dataX.append(a)
        dataY.append(dataset[i, -1])
    return numpy.array(dataX), numpy.array(dataY)

numpy.random.seed(7)

dataframe = pd.read_csv('./ML_cols.csv', usecols=[1,2,3,4,5])
dataset = dataframe.values
dataset = dataset.astype('float32')
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# trainX = x_scaler.fit_transform(trainX)
# trainY  = y_scaler.fit_transform(trainY.reshape(-1,1))
# testX = x_scaler.transform(testX)
# testY = y_scaler.transform(testY.reshape(-1,1))

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

model = Sequential()
model.add(LSTM(input_shape=(4 * look_back, trainX.shape[2]), return_sequences=True, units=50))
model.add(Dropout(0.5))
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(1))
#model.add(LeakyReLU(alpha=0.3))
model.add(Activation('linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=30, batch_size=1, verbose=2)


Epoch 1/30
 - 8s - loss: 0.0548
Epoch 2/30
 - 3s - loss: 0.0309
Epoch 3/30
 - 3s - loss: 0.0321
Epoch 4/30
 - 3s - loss: 0.0281
Epoch 5/30
 - 3s - loss: 0.0281
Epoch 6/30
 - 3s - loss: 0.0188
Epoch 7/30
 - 3s - loss: 0.0268
Epoch 8/30
 - 3s - loss: 0.0234
Epoch 9/30
 - 4s - loss: 0.0210
Epoch 10/30
 - 4s - loss: 0.0221
Epoch 11/30
 - 4s - loss: 0.0205
Epoch 12/30
 - 4s - loss: 0.0214
Epoch 13/30
 - 4s - loss: 0.0188
Epoch 14/30
 - 4s - loss: 0.0190
Epoch 15/30
 - 4s - loss: 0.0222
Epoch 16/30
 - 3s - loss: 0.0215
Epoch 17/30
 - 4s - loss: 0.0176
Epoch 18/30
 - 4s - loss: 0.0183
Epoch 19/30
 - 4s - loss: 0.0194
Epoch 20/30
 - 5s - loss: 0.0201
Epoch 21/30
 - 5s - loss: 0.0182
Epoch 22/30
 - 4s - loss: 0.0193
Epoch 23/30
 - 4s - loss: 0.0200
Epoch 24/30
 - 4s - loss: 0.0206
Epoch 25/30
 - 4s - loss: 0.0182
Epoch 26/30
 - 3s - loss: 0.0187
Epoch 27/30
 - 3s - loss: 0.0202
Epoch 28/30
 - 3s - loss: 0.0194
Epoch 29/30
 - 3s - loss: 0.0168
Epoch 30/30
 - 4s - loss: 0.0196


In [53]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainCorrect = trainY
testCorrect = testY
# trainPredict = y_scaler.inverse_transform(trainPredict)
# trainCorrect = y_scaler.inverse_transform(trainY)
# testPredict = y_scaler.inverse_transform(testPredict)
# testCorrect = y_scaler.inverse_transform(testY)
import math

def sign(a):
    if a>0:
        return 1
    if a<0:
        return -1
    return 0

def acc(a, b):
    cnt = 0
    for i in range(len(a)):
        print(a[i], b[i])
        if sign(a[i]) == sign(b[i]):
            cnt += 1
    return
    return cnt / len(a)

trainScore = math.sqrt(mean_squared_error(trainCorrect[:], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
print()
print("acc: ", acc((trainCorrect[:]), (trainPredict[:,0])))
print()
print()
print()
testScore = math.sqrt(mean_squared_error(testCorrect[:], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
print()
print("acc: ", acc((testCorrect[:]), (testPredict[:,0])))


Train Score: 0.12 RMSE

0.67892647 0.6203965
0.5478052 0.6196151
0.61476463 0.6192994
0.5737857 0.6209295
0.98283684 0.6187185
0.54161435 0.62017155
0.6445639 0.61439764
0.7576961 0.6219719
0.6721982 0.61820567
0.7707161 0.6177963
0.815034 0.61889786
0.73475665 0.61724913
0.54184175 0.61279345
0.76299846 0.61830723
0.80813175 0.6201404
0.6614212 0.6183326
0.7487795 0.6179949
0.7051129 0.6185205
0.7196996 0.61790705
0.76028764 0.6186055
0.6564791 0.6178532
0.5021078 0.616778
0.4524821 0.61874664
0.7007516 0.6199395
0.64141214 0.6201685
0.75115144 0.6194309
0.7437887 0.62035215
0.658672 0.6176226
0.82839406 0.61849964
0.847597 0.61913073
0.95513767 0.61488163
0.6869317 0.61455613
0.90811497 0.61481464
0.69363004 0.6165501
0.6738591 0.6121253
0.684631 0.61717355
0.72573876 0.617201
0.6013743 0.6175517
0.613533 0.61625385
0.5723605 0.6189774
0.6951423 0.6193963
0.5897839 0.6186137
0.6114612 0.61701787
0.68146753 0.6197145
0.61613554 0.61966413
0.6742192 0.6190071
0.62215525 0.6197245
0.644